In [31]:
# Imports

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup # переводит данные в более понятный формат
import html5lib

import pandas as pd
import time
import random

In [37]:
# Variables

first_url = 'https://sbermegamarket.ru/' # не менять
city = "Казань" # город доставки
url = 'https://sbermegamarket.ru/catalog/smartfony-xiaomi/' # каталог с интересующими товарами
max_page=20 # количество просматриваемых страниц, не более 40

C:\Users\1\AppData\Local\Temp\ipykernel_316\1543065976.py:13: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=option)


In [32]:
# My Functions: запуск первой страницы

def go_start_v1(url=first_url, browser=0):
    if browser == 0:

        option = Options()
        option.add_argument("--disable-infobars")
        option.add_argument('--disable-notifications')
        option.add_experimental_option(
            "prefs", {"profile.default_content_setting_values.notifications": 2}
            )

        browser = webdriver.Chrome(chrome_options=option)
        
    browser.get(url)
    time.sleep(3)
    browser.find_element(By.CLASS_NAME, "svg-icon").click()
    time.sleep(1)
    browser.find_element(By.CLASS_NAME, "icon").click()
    time.sleep(1)
    browser.find_element(By.CLASS_NAME, "mobile-region-selector__pseudo-select").click()
    time.sleep(1)
    browser.find_element(By.CLASS_NAME, "header-region-selector-view__change-link").click()
    time.sleep(1)

    form = browser.find_element(By.TAG_NAME, "input")
    submit = browser.find_element(By.CSS_SELECTOR, "#app > div:nth-child(4) > div > div > div > div > div > div > div > div > form > button")

    actions = ActionChains(browser)
    actions.move_to_element(form)
    actions.click()
    actions.send_keys("Казань")
    actions.pause(1)
    actions.send_keys(Keys.DOWN)
    actions.send_keys(Keys.ENTER)
    actions.click(submit)
    #actions.send_keys(Keys.ESCAPE)
    actions.perform()
    return browser

In [33]:
# My Functions: запуск первой страницы (not work)

def go_start_v2(url, browser=0):

    if browser == 0:
        option = Options()

        option.add_argument("--disable-infobars")
        option.add_argument("start-maximized")
        option.add_argument("--disable-extensions")

        # Pass the argument 1 to allow and 2 to block
        option.add_experimental_option(
            "prefs", {"profile.default_content_setting_values.notifications": 2}
        )

        browser = webdriver.Chrome(
            chrome_options=option
            ) #, executable_path="path-of-driver\chromedriver.exe"

    browser.get(url)

    time.sleep(5)

    css = 'btn-bordered header-region-selector-view__footer-cancel xs'
    browser.find_element(By.CLASS_NAME, css)

    css = '#page-header > div > div.links-container > div > div.contacts > div > div > div > div > div > div > form > div > div > div > input'
    form = browser.find_element(By.CSS_SELECTOR, css)
    css = "#page-header > div > div.links-container > div > div.contacts > div > div > div > div > div > div > form > button"
    submit = browser.find_element(By.CSS_SELECTOR, css)

    actions = ActionChains(browser)
    actions.move_to_element(form)
    actions.click()
    actions.send_keys(city)
    actions.pause(1)
    actions.send_keys(Keys.DOWN)
    actions.send_keys(Keys.ENTER)
    actions.click(submit)
    #actions.send_keys(Keys.ESCAPE)
    actions.perform()

    return browser

In [34]:
# My function: информация о продукте

def get_info(item):
    title = item.find('div', class_='item-title').text.replace('\t', '').replace('\n', '')
    price = int(item.find('div', class_='item-price').text.replace('\xa0₽', '').replace(' ', ''))
    bonus = int(item.find('span', 'bonus-amount').text.replace(' ', ''))
    site = first_url + item.find('a')['href']
    perc = bonus/price*100
    return title, price, bonus, perc, site

In [35]:
def go_parse(url, driver, max_page, i=0):
    driver.get(url) # заходим на страницу
    
    # глобальные переменные
    global df

     # создаем Soup
    pageSource = driver.execute_script("return document.body.innerHTML;")
    soup = BeautifulSoup(pageSource, 'html5lib')

    # находим блок кода с информацией о продуктах
    div = soup.find_all('div', class_='catalog-item ddl_product')

    # собираем информацию о продуктах
    data = []
    for item in div:
        data.append(get_info(item))

    # сохраняем информацию в датафрейм
    try:
        df
    except NameError:
        df = pd.DataFrame(columns=['item', 'price', 'bonus', 'percent', 'site'])
    
    df = pd.concat([df, pd.DataFrame(data, columns=df.columns)], ignore_index=True)
    
    # проверяем, есть ли другие страницы и парсим ссылку
    if str(soup).find('class="next"') != -1:
        next = soup.find_all('li', 'next')
        url_next = first_url + str(next).split('href="', 1)[1].split('"', 1)[0]
    else:
        return df
    
    # эмулируем работу на странице
    ActionChains(driver)\
        .send_keys(Keys.PAGE_DOWN)\
        .send_keys(Keys.PAGE_DOWN)\
        .pause(1)\
        .send_keys(Keys.PAGE_DOWN)\
        .send_keys(Keys.PAGE_DOWN)\
        .send_keys(Keys.PAGE_DOWN)\
        .perform()

    # задержка от блокировки
    time.sleep(random.uniform(2,5))

    # заход на следующий круг

    try:
        max_page
    except NameError:
        max_page = 19

    if i < max_page:
        i += 1
        go_parse(url_next, driver, max_page, i=i)
    else:
        return df
    
    # возвращаем результат работы
    return df


In [36]:
def main(url, max_page=19, driver=0):
    driver = go_start_v1(browser=driver)
    df = go_parse(url=url, driver=driver, max_page=max_page)
    return df

In [38]:
# получаем датафрейм с лучшими предложениями
df = main(url, max_page)
df

,item,price,bonus,percent,site
0,Смартфон POCO X5 Pro 5G 8/256GB Black,29690,1485,5.001684,https://sbermegamarket.ru//catalog/details/sma...
1,Смартфон Xiaomi Redmi Note 10 Pro 8/128GB Onyx...,23999,240,1.000042,https://sbermegamarket.ru//catalog/details/sma...
2,Смартфон Xiaomi Redmi 10C 4/128GB Gray (38594),12770,128,1.002349,https://sbermegamarket.ru//catalog/details/sma...
3,Смартфон Xiaomi Redmi A1+ 2/32GB Black (43094),5990,60,1.001669,https://sbermegamarket.ru//catalog/details/sma...
4,Смартфон Xiaomi Redmi Note 11 Pro 5G 8/128GB G...,24990,2500,10.004002,https://sbermegamarket.ru//catalog/details/pda...
...,...,...,...,...,...
499,Смартфон Xiaomi 11T 8/256GB Meteorite Gray (21...,42276,423,1.000568,https://sbermegamarket.ru//catalog/details/sma...
500,Смартфон Xiaomi Mi 11T Pro 5G 8/256Gb Blue (EU),41475,415,1.000603,https://sbermegamarket.ru//catalog/details/sma...
501,Смартфон Xiaomi Redmi 9C 4/128GB Lavender,14479,145,1.001450,https://sbermegamarket.ru//catalog/details/xia...
502,Смартфон Xiaomi Redmi Note 11 Pro 6/128Gb 4G W...,32720,328,1.002445,https://sbermegamarket.ru//catalog/details/sma...


In [41]:
# отбираем только интересующие модели и сортируем по проценту бонуса
result = df.copy()
result = df[df['item'].str.contains("Xiaomi")].sort_values('percent', ascending=False)
result

,item,price,bonus,percent,site
186,Смартфон Xiaomi 12S Ultra 12/256Gb (CN) Green ...,91718,18344,20.000436,https://sbermegamarket.ru//catalog/details/sma...
173,Смартфон Xiaomi Redmi 10A 2/32GB Sky Blue,6480,1296,20.000000,https://sbermegamarket.ru//catalog/details/sma...
471,"Смартфон Xiaomi POCO M5 4/64 ГБ, желтый",13367,1472,11.012194,https://sbermegamarket.ru//catalog/details/sma...
440,Смартфон Xiaomi Redmi Note 11 4/64GB Graphite ...,16471,1813,11.007225,https://sbermegamarket.ru//catalog/details/xia...
280,Смартфон Xiaomi Mi 12 8/256GB Blue (art-13161),51061,5618,11.002526,https://sbermegamarket.ru//catalog/details/sma...
...,...,...,...,...,...
446,Смартфон Xiaomi 11 Lite 5G NE 8/256Gb White (EU),37500,375,1.000000,https://sbermegamarket.ru//catalog/details/sma...
359,Смартфон Xiaomi Poco C40 4/64GB Coral Green EU...,9400,94,1.000000,https://sbermegamarket.ru//catalog/details/xia...
153,Смартфон Xiaomi Redmi 9A 2/32GB Granite Gray,5300,53,1.000000,https://sbermegamarket.ru//catalog/details/sma...
422,Смартфон Xiaomi Poco M5S 4/128Gb Grey EU NFC,15900,159,1.000000,https://sbermegamarket.ru//catalog/details/sma...


In [42]:
# получаем ссылку на заинтересовавшее предложение
result.iloc[0,4]

'https://sbermegamarket.ru//catalog/details/smartfon-xiaomi-12s-ultra-12-256gb-cn-green-zelenyy-100044879086/'